# ***`Homework`***
---

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
jan23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## ***Q1. Downloading the data***
---

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

In [3]:
print(f"Number of columns: {jan23.shape[1]}")

Number of columns: 19


## Q2. Computing duration
---

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

In [4]:
jan23['duration_minutes'] = (jan23['tpep_dropoff_datetime'] - jan23['tpep_pickup_datetime']).dt.total_seconds() / 60
jan23['duration_minutes'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration_minutes, dtype: float64

In [5]:
# do same to validation data set (feb23)
feb23['duration_minutes'] = (feb23['tpep_dropoff_datetime'] - feb23['tpep_pickup_datetime']).dt.total_seconds() / 60

## ***Q3. Dropping outliers***
---

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [6]:
mask = (jan23['duration_minutes'] >= 1) & (jan23['duration_minutes'] <= 60)

# Filter out trips that are less than 1 minute and more than 60 minutes
jan23_clean = jan23[mask].copy()

# fraction
fraction = jan23_clean.shape[0] / jan23.shape[0]

print(f"Fraction of trips that are between 1 and 60 minutes: {fraction:.2f}")

Fraction of trips that are between 1 and 60 minutes: 0.98


In [7]:
# do same to validation data set (feb23)
mask = (feb23['duration_minutes'] >= 1) & (feb23['duration_minutes'] <= 60)
feb23_clean = feb23[mask].copy()

## ***Q4. One-hot encoding***
---

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

In [8]:
from sklearn.feature_extraction import DictVectorizer

# filter columns:
columns     = ['PULocationID', 'DOLocationID', 'duration_minutes']
jan23_clean = jan23_clean[columns]
feb23_clean = feb23_clean[columns]

# cast to str
jan23_clean['PULocationID'] = jan23_clean['PULocationID'].astype(str)
jan23_clean['DOLocationID'] = jan23_clean['DOLocationID'].astype(str)

feb23_clean['PULocationID'] = feb23_clean['PULocationID'].astype(str)
feb23_clean['DOLocationID'] = feb23_clean['DOLocationID'].astype(str)

# define train a. validation sets
X_train, y_train = jan23_clean.drop(columns='duration_minutes'), jan23_clean['duration_minutes']
X_test, y_test   = feb23_clean.drop(columns='duration_minutes'), feb23_clean['duration_minutes']

# vectorize
dv = DictVectorizer(sparse=False)

train_dict      = X_train.to_dict(orient='records')
X_train_encoded = dv.fit_transform(train_dict)

test_dict       = X_test.to_dict(orient='records')
X_test_encoded  = dv.transform(test_dict)

# n dimensions
print(f"Number of columns: {X_train_encoded.shape[1]}")

Number of columns: 515


In [9]:
X_train_encoded[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## ***Q5. Training a model***
---

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

rmse = lambda y_true, y_pred: np.sqrt( mean_squared_error(y_true, y_pred) )

# train model
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# predict
y_pred_train = model.predict(X_train_encoded)

# evaluate
rmse(y_train, y_pred_train)

## ***Q6. Evaluating the model***
---

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

In [ ]:
y_pred_test = model.predict(X_test_encoded)
rmse(y_test, y_pred_test)